In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
# clone data https://github.com/waittim/mask-detector.git
!git clone https://github.com/waittim/mask-detector.git

Cloning into 'mask-detector'...
remote: Enumerating objects: 36671, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 36671 (delta 48), reused 110 (delta 11), pack-reused 36498
Receiving objects: 100% (36671/36671), 1.94 GiB | 34.63 MiB/s, done.
Resolving deltas: 100% (8139/8139), done.
Checking out files: 100% (19611/19611), done.


In [ ]:
# clone ssd https://github.com/qfgaohao/pytorch-ssd.git
!git clone https://github.com/qfgaohao/pytorch-ssd.git

Cloning into 'pytorch-ssd'...
remote: Enumerating objects: 812, done.
remote: Total 812 (delta 0), reused 0 (delta 0), pack-reused 812
Receiving objects: 100% (812/812), 1.05 MiB | 29.06 MiB/s, done.
Resolving deltas: 100% (544/544), done.


In [ ]:
# CONVERTOR https://github.com/carolinepacheco/convert-yolo-to-pascalvoc
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Aug 17 22:10:01 2018

@author: Caroline Pacheco do E. Silva
"""

import os
import cv2
from xml.dom.minidom import parseString
from lxml.etree import Element, SubElement, tostring
import numpy as np
from os.path import join

## coco classes
YOLO_CLASSES = ('no_mask', 'with_mask')


## converts the normalized positions  into integer positions
def unconvert(class_id, width, height, x, y, w, h):

    xmax = int((x*width) + (w * width)/2.0)
    xmin = int((x*width) - (w * width)/2.0)
    ymax = int((y*height) + (h * height)/2.0)
    ymin = int((y*height) - (h * height)/2.0)
    class_id = int(class_id)
    return (class_id, xmin, xmax, ymin, ymax)


## path root folder
ROOT = '/content/mask-detector/modeling/data'


## converts coco into xml 
def xml_transform(root, classes):  
    class_path  = join(root, 'labels')
    ids = list()
    l=os.listdir(class_path)
    
    check = '.DS_Store' in l
    if check == True:
        l.remove('.DS_Store')
        
    ids=[x.split('.')[0] for x in l]   

    annopath = join(root, 'labels', '%s.txt')
    imgpath = join(root, 'images', '%s.jpg')
    
    os.makedirs(join(root, 'outputs'), exist_ok=True)
    outpath = join(root, 'outputs', '%s.xml')

    for i in range(len(ids)):
        img_id = ids[i] 
        img= cv2.imread(imgpath % img_id)
        height, width, channels = img.shape # pega tamanhos e canais das images

        node_root = Element('annotation')
        node_folder = SubElement(node_root, 'folder')
        node_folder.text = 'VOC2007'
        img_name = img_id + '.jpg'
    
        node_filename = SubElement(node_root, 'filename')
        node_filename.text = img_name
        
        node_source= SubElement(node_root, 'source')
        node_database = SubElement(node_source, 'database')
        node_database.text = 'Coco database'
        
        node_size = SubElement(node_root, 'size')
        node_width = SubElement(node_size, 'width')
        node_width.text = str(width)
    
        node_height = SubElement(node_size, 'height')
        node_height.text = str(height)

        node_depth = SubElement(node_size, 'depth')
        node_depth.text = str(channels)

        node_segmented = SubElement(node_root, 'segmented')
        node_segmented.text = '0'

        target = (annopath % img_id)
        if os.path.exists(target):
            label_norm= np.loadtxt(target).reshape(-1, 5)

            for i in range(len(label_norm)):
                labels_conv = label_norm[i]
                new_label = unconvert(labels_conv[0], width, height, labels_conv[1], labels_conv[2], labels_conv[3], labels_conv[4])
                node_object = SubElement(node_root, 'object')
                node_name = SubElement(node_object, 'name')
                node_name.text = classes[new_label[0]]
                
                node_pose = SubElement(node_object, 'pose')
                node_pose.text = 'Unspecified'
                
                node_truncated = SubElement(node_object, 'truncated')
                node_truncated.text = '0'
                node_difficult = SubElement(node_object, 'difficult')
                node_difficult.text = '0'
                node_bndbox = SubElement(node_object, 'bndbox')
                node_xmin = SubElement(node_bndbox, 'xmin')
                node_xmin.text = str(new_label[1])
                node_ymin = SubElement(node_bndbox, 'ymin')
                node_ymin.text = str(new_label[3])
                node_xmax = SubElement(node_bndbox, 'xmax')
                node_xmax.text =  str(new_label[2])
                node_ymax = SubElement(node_bndbox, 'ymax')
                node_ymax.text = str(new_label[4])
                xml = tostring(node_root, pretty_print=True)  
                dom = parseString(xml)
        # print(xml)  
        f =  open(outpath % img_id, "wb")
        #f = open(os.path.join(outpath, img_id), "w")
        #os.remove(target)
        f.write(xml)
        f.close()     
       

xml_transform(ROOT, YOLO_CLASSES)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: UserWarning: loadtxt: Empty input file: "/content/mask-detector/modeling/data/labels/test_00001352.txt"
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: UserWarning: loadtxt: Empty input file: "/content/mask-detector/modeling/data/labels/41_Swimming_Swimmer_41_174.txt"


In [ ]:
%mkdir mask_data
%mkdir mask_data/Annotations
%mkdir mask_data/JPEGImages
%mkdir mask_data/ImageSets
%mkdir mask_data/ImageSets/Main

!cp mask-detector/modeling/data/images/*.jpg mask_data/JPEGImages
!cp mask-detector/modeling/data/outputs/*.xml mask_data/Annotations

In [ ]:
def get_file_name(path):
    file_name_ex = os.path.split(path)[1]
    file_name = os.path.splitext(file_name_ex)[0]

    return file_name

In [ ]:
import glob 

train_paths = glob.glob('mask_data/JPEGImages/*.jpg')[:7000]
with open('mask_data/ImageSets/Main/trainval.txt', 'w') as f:
  f.write('\n'.join([get_file_name(i) for i in train_paths]))

val_paths = glob.glob('mask_data/JPEGImages/*.jpg')[7000:]
with open('mask_data/ImageSets/Main/test.txt', 'w') as f:
  f.write('\n'.join([get_file_name(i) for i in val_paths]))

with open('mask_data/labels.txt', 'w') as f:
  f.write('no_mask,\nwith_mask')

In [ ]:
import numpy as np

np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)
# paste that line in vision/transforms/transforms.py 
# file here: https://github.com/qfgaohao/pytorch-ssd/blob/master/vision/transforms/transforms.py

In [ ]:
import time 
t = time.localtime()
current_time = time.strftime('%y_%m_%d_%H_%M_%S', t)
checkpoint_path = '/content/drive/MyDrive/mask_detect_ssd/' + current_time
os.mkdir(checkpoint_path)

In [ ]:
!wget -P models https://storage.googleapis.com/models-hao/mb2-imagenet-71_8.pth
!python pytorch-ssd/train_ssd.py --dataset_type voc  --datasets mask_data --validation_dataset mask_data --net mb2-ssd-lite --base_net models/mb2-imagenet-71_8.pth  --scheduler cosine --lr 0.01 --t_max 200 --validation_epochs 5 --num_epochs 200 --checkpoint_folder $checkpoint_path

--2022-05-19 01:37:25--  https://storage.googleapis.com/models-hao/mb2-imagenet-71_8.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.211.128, 173.194.212.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.211.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9067303 (8.6M) [application/octet-stream]
Saving to: ‘models/mb2-imagenet-71_8.pth’

mb2-imagenet-71_8.p 100%[===================>]   8.65M  --.-KB/s    in 0.1s    

2022-05-19 01:37:25 (87.8 MB/s) - ‘models/mb2-imagenet-71_8.pth’ saved [9067303/9067303]

2022-05-19 01:37:29,092 - root - INFO - Use Cuda.
2022-05-19 01:37:29,093 - root - INFO - Namespace(balance_data=False, base_net='models/mb2-imagenet-71_8.pth', base_net_lr=None, batch_size=32, checkpoint_folder='/content/drive/MyDrive/mask_detect_ssd/22_05_19_01_37_24', dataset_type='voc', datasets=['mask_data'], debug_steps=100, extra_layers_lr=None, freeze_base_net=False, freeze_ne

In [ ]:
!wget -P models https://storage.googleapis.com/models-hao/vgg16-ssd-mp-0_7726.pth
!python pytorch-ssd/train_ssd.py --datasets mask_data --validation_dataset mask_data --net vgg16-ssd --pretrained_ssd models/vgg16-ssd-mp-0_7726.pth  --batch_size 24 --num_epochs 200 --scheduler "multi-step" --milestones "120,160"

--2022-05-19 05:56:28--  https://storage.googleapis.com/models-hao/vgg16-ssd-mp-0_7726.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.63.128, 142.250.31.128, 142.250.188.208, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.63.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 105162133 (100M) [application/octet-stream]
Saving to: ‘models/vgg16-ssd-mp-0_7726.pth’

vgg16-ssd-mp-0_7726 100%[===================>] 100.29M   132MB/s    in 0.8s    

2022-05-19 05:56:29 (132 MB/s) - ‘models/vgg16-ssd-mp-0_7726.pth’ saved [105162133/105162133]

2022-05-19 05:56:33,542 - root - INFO - Namespace(balance_data=False, base_net=None, base_net_lr=None, batch_size=24, checkpoint_folder='models/', dataset_type='voc', datasets=['mask_data'], debug_steps=100, extra_layers_lr=None, freeze_base_net=False, freeze_net=False, gamma=0.1, lr=0.001, mb2_width_mult=1.0, milestones='120,160', momentum=0.9, net='vgg16-ssd', num_epochs=200